In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/CarParts_YOLO26'
os.makedirs(drive_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.7 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import shutil

# 1. Load the latest YOLO26 segmentation model
model = YOLO('yolo26n-seg.pt')

# 2. Define the rewriting logic (Auto-Save every 5 epochs)
def backup_callback(trainer):
    epoch = trainer.epoch + 1
    if epoch % 5 == 0:
        source = f"{trainer.save_dir}/weights/best.pt"
        dest = f"{drive_path}/yolo26_carparts_latest.pt"
        if os.path.exists(source):
            shutil.copy(source, dest)
            print(f"\n[Drive] Epoch {epoch}: Overwrote backup at {dest}")

model.add_callback("on_train_epoch_end", backup_callback)

# 3. Train for 100 epochs
model.train(data='carparts-seg.yaml', epochs=100, imgsz=640, project='CarProject', name='YOLO26_Run')

Ultralytics 8.4.18 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=carparts-seg.yaml, degrees=0.0, deterministic=True, device=6,7, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo26n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=YOLO26_Run2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0

KeyboardInterrupt: 

In [ ]:
import os
import shutil
from ultralytics import YOLO
from google.colab import drive

# 1. Mount Drive and Define Path
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/CarParts_YOLO26'
checkpoint = f"{drive_path}/yolo26_carparts_latest.pt"

# 2. Load the checkpoint
model = YOLO(checkpoint)

# 3. Re-add Callback
def backup_callback(trainer):
    epoch = trainer.epoch + 1
    if epoch % 5 == 0:
        best_src = f"{trainer.save_dir}/weights/best.pt"
        last_src = f"{trainer.save_dir}/weights/last.pt"
        if os.path.exists(best_src):
            shutil.copy(best_src, f"{drive_path}/yolo26_carparts_best.pt")
        if os.path.exists(last_src):
            shutil.copy(last_src, f"{drive_path}/yolo26_carparts_latest.pt")
        print(f"\n[Drive] Epoch {epoch}: Backups updated.")

model.add_callback("on_train_epoch_end", backup_callback)

# 4. Resume Training - Added device=0 to fix the DDP crash
# Ensure 'carparts-seg.yaml' is in your current working directory
model.train(resume=True, device=0)

Ultralytics 8.4.18 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/datasets/carparts-seg.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/drive/MyDrive/CarParts_YOLO26/yolo26_carparts_latest.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=YOLO26_Ru

KeyboardInterrupt: 

In [ ]:
from ultralytics import YOLO
import os

# 1. Path to your best model
drive_path = '/content/drive/MyDrive/CarParts_YOLO26'
model_path = f"{drive_path}/yolo26_carparts_best.pt"

if os.path.exists(model_path):
    # 2. Load the model
    model = YOLO(model_path)

    # 3. Access the training metrics stored in the checkpoint
    # Note: YOLO saves the best fitness and individual metrics in the ckpt dictionary
    if hasattr(model, 'ckpt') and 'train_metrics' in model.ckpt:
        metrics = model.ckpt['train_metrics']
        print("--- Best Model Performance Scores ---")
        for key, value in metrics.items():
            print(f"{key}: {value:.4f}")
    else:
        print("No specific training metrics found inside this checkpoint. You may need to run model.val() to get the current scores.")

        # Optional: Run a quick validation to get fresh scores
        # results = model.val(data='carparts-seg.yaml')
        # print(results.results_dict)
else:
    print(f"File not found at: {model_path}")

--- Best Model Performance Scores ---
metrics/precision(B): 0.5968
metrics/recall(B): 0.7800
metrics/mAP50(B): 0.6754
metrics/mAP50-95(B): 0.5632
metrics/precision(M): 0.5816
metrics/recall(M): 0.7878
metrics/mAP50(M): 0.6773
metrics/mAP50-95(M): 0.5469
val/box_loss: 0.7069
val/seg_loss: 1.0041
val/cls_loss: 0.5982
val/dfl_loss: 0.0076
val/sem_loss: 0.0000
fitness: 1.1101


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Convert to Android-friendly format
model.export(format='tflite')

# Download the tflite file
files.download('runs/segment/train/weights/best_float32.tflite')